# For a mab ID if there is a droupout but the same mab ID has a good sequence we dont care about them
- summary of HC or just LC as well.. per plate stats. 
- if no shown on web then a Dec dropout is ok
- up to the first dot can be the same (D4/81.1, D4/81.2)
- dont rely on columns from the duplicates too much for dropout etc. 
- generate all the information as well


# Duplications and subclones
- project name, parent clone number, subclone number (sibilings)
- for every clone for it to have a cooraborating sequence in the database. If .1 has sequence than we want .2,.3 etc to have the same sequence. 
- what number of the duplicates for an ASV have the same parent so are subclones
- 49 clones with 47 parents... is it real? What plates are these from?
    - some other sequences that keeps showing up but doesnt 
- same subclone but one is off.. red - one typo but two 
- which are ok? Which need to be binned or need more analysis, which are close on the plate
- do some have sequences with multiple chains across multiple ASVs. 

- ASV amount
- eventually deactivating them?
- across plates or across wells
- **coding should match undergrad**
- **if on the same plate could be a contaminant... different plate typo?**


# How will this work better in the future
- what happens if a trimmer id is changed
- DONT USE THE TIMMER ID IN THE DROPOUT FILE OR THE TRIMMERID IN THE DUPLICATE FILE BECAUSE WHAT IF THEY CHANGE IN THE METADATA


# Sequence deactivation
- what if we can get rid of certain sequences... so like duplicates with one sketch HC but a chill LC
- 

# Need to find out the categories to make sure cooborating sequences for category 1 and 3
- combine duplicate HC and LC analysis to make sure they are in duplicate categories that are good (1 and 2)


# HEAVY CHAINS

In [129]:
import pandas as pd
# why does the MabID column have different number of values then the duplicated in
data = pd.read_csv('DuplicatedHC_ASVs.tsv', sep='\t')
data.tail()

,ASV,DuplicatedIn,MabIDs,ParentIDs,MabCount,ParentCount
1252,ACATCGCTCTCACTGGAGGCTGATCTCTGAAGATAAGGAGGTGTAG...,"P65_C10, P78_C10","N443/15.1, N443/15.3",N443/15,2,1
1253,ACATCGCTCTCACTGGAGGCTGATCTCTGAAGATAAGGAGGTGTAG...,"P29_H6, P78_G10, P78_G11","N459/90.1, N459/90.2, N459/90.5",N459/90,3,1
1254,ACAGTCACTGAAAACATTGACTCTAATCATGGAATGTAACTGGATA...,"P29_H5, P78_B11","N459/11.1, N459/11.6",N459/11,2,1
1255,ATATGAGCCCTATCTTCTCTACAGACACTGAATCTCAAGGTCCTTA...,"P31_B5, P78_C11","N439/39.1, N439/39.3",N439/39,2,1
1256,ACAAGTGTGCAGCCATGGGCAGGCTTACTTCTTCATTCCTGCTACT...,"P33_D3, P78_G12","N438/98.1, N438/98.12",N438/98,2,1


In [130]:
# upload the metadata
dropout = pd.read_csv("alldata_master.tsv", sep='\t')
dropout.head()

,sample_name,trimmer_id,PlateName,Category,ShowOnWeb,ProteinTarget,ChangeDate,ChangeReason
0,None,SP2/0,RandD_samples,NaN,F,NaN,20200428.0,Control
1,P1_A1,positivecontrol1_N263/31.1,plate1,NaN,F,NaN,20201009.0,NaN
2,P1_A10,N53/32.1,plate1,1.0,F,BKbeta2 K+ channel,20201009.0,Plate failed to yield seq so redone as Plate1R
3,P1_A11,N57/2.4,plate1,1.0,F,ADAM22,20201009.0,Plate failed to yield seq so redone as Plate1R
4,P1_A12,N72/16.1,plate1,1.0,F,Kv3.4 K+ channel,20201009.0,Plate failed to yield seq so redone as Plate1R


In [131]:
metadata2 = pd.read_csv("alldata_master.tsv", sep='\t')
print(len(metadata2))
meta2 = metadata2[metadata2["ShowOnWeb"]=="T"]
print(len(meta2))

meta1 = dropout[dropout["ShowOnWeb"]=="T"]
print(len(meta1))

result = list(set(meta1['sample_name']) - set(meta2['sample_name']))
print(result)
print(len(result))
print(set([i.split('_')[0] for i in result]))

7559
6272
6272
[]
0
set()


In [132]:
bool_series = dropout.sample_name.isin(result)
dropout[bool_series]
dropout[bool_series].to_csv('metadata_difference.tsv')

    
# Duplicates
## -1: These did not work in the code
- likely due to some having subclone in one but not another (TODO write new rule for this)
- also occurs when no / in the ID
- often positive control ones


## 0 CATEGORY: These are ok. No sublone in them
- check if all entries have no dots.. do not include in subsequent analysis

## 1 CATEGORY: These are ok as well. All IDs match.
- check if all the same

## 2 CATEGORY: just the subclone
- check if all same up to the subclone type (up to the first dot) (projects and parents all the same but cant be all same)


## 3 CATEGORY: likely typos with groups more then 2    
- check if project is the same but parent is different and subclone is the same (indicates typo in parent)
- 3

- check if project is different but parent is same and subclone is the same (indicates typo in project)
- 3.5

- check if project is different and parent is different and subclone is the same (indicates typo in parent and subclone)
- 3.6

## 4 CATEGORY: same as 3 but should only be for 2 entry groups
- check if project is the same but parent is different and subclone is the same (indicates typo in parent)
- 4

- check if project is different but parent is same and subclone is the same (indicates typo in project)
- 4.5

- check if project is different and parent is different and subclone is the same (indicates typo in parent and subclone)
- 4.6


## 5 CATEGORY: bad?

In [133]:
def plate_to_mab(plate_loc):
    try:
        #if list(entry['trimmer_id'])[0]:
        entry = dropout[dropout['sample_name']==plate_loc]
        return(list(entry['trimmer_id'])[0])
    except:
        print("ERROR: there is an issue with %s" %(plate_loc))
plate_to_mab('P41_A10')

'N187/33.1'

In [134]:
def get_category(mydata):
    mycatlist = []
    myparentlist = []
    mysubclonelist = []
    for index,row in mydata.iterrows():
        # todo should be the duplicated in and grab the mabID
        #print(row['DuplicatedIn'])
        print(row['DuplicatedIn'])
        meta_listupdate = [plate_to_mab(i.replace(" ","")) for i in row['DuplicatedIn'].split(',')] 
        print(meta_listupdate)
        mab_id_list = [i.replace(' ','') for i in meta_listupdate if i]
        #print([i.split('.')[0].replace(' ','') for i in mab_id_list])
        #print(len(set([i.split('.')[0].replace(' ','') for i in mab_id_list])))

        try:
            projects = [i.split('/')[0].replace(' ','') for i in mab_id_list]
            parents = [i.split('/')[1].split('.')[0].replace(' ', '') for i in mab_id_list]
            subclone = [i.split('/')[1].split('.')[1].replace(' ', '') for i in mab_id_list]
            #print(projects)
            #print(parents)
            #print(subclone)    
            myparentlist.append(parents)
            mysubclonelist.append(mab_id_list)


            # 0 CATEGORY: These are ok. No sublone in them
            # check if all entries have no dots.. do not include in subsequent analysis
            if sum([1 for i in mab_id_list if '.' in i])<1:
                mycatlist.append(0)    

            # 1 CATEGORY: These are ok as well. All IDs match.
            # check if all the same
            elif len(set(mab_id_list))==1:
                mycatlist.append(1)

            # 2 CATEGORY: just the subclone
            # check if all same up to the subclone type (up to the first dot) (projects and parents all 
            # the same but cant be all same)
            elif len(set(projects))==1 and len(set(parents))==1:
                mycatlist.append(2)


            # 3 CATEGORY: likely typos with groups more then 2    
            # check if project is the same but parent is different and subclone is the same (indicates typo in parent)
            elif len(set(projects))==1 and len(set(parents))==2 and len(set(subclone))==1 and len(mab_id_list)>2:
                mycatlist.append(3)

            # check if project is different but parent is same and subclone is the same (indicates typo in parent)
            elif len(set(projects))==2 and len(set(parents))==1 and len(set(subclone))==1 and len(mab_id_list)>2:
                mycatlist.append(3.5)

            # check if project is different and parent is different and subclone is the same (indicates typo in parent and subclone)
            elif len(set(projects))==1 and len(set(parents))==2 and len(set(subclone))==2 and len(mab_id_list)>2:
                mycatlist.append(3.6)


            # 4 CATEGORY: same as 3 but no requirement for more then 2 entries
            # check if project is the same but parent is different and subclone is the same (indicates typo in parent)
            elif len(set(projects))==1 and len(set(parents))==2 and len(set(subclone))==1:
                mycatlist.append(4)

            # check if project is different but parent is same and subclone is the same (indicates typo in parent)
            elif len(set(projects))==2 and len(set(parents))==1 and len(set(subclone))==1:
                mycatlist.append(4.5)

            # check if project is different and parent is different and subclone is the same (indicates typo in parent and subclone)
            elif len(set(projects))==1 and len(set(parents))==2 and len(set(subclone))==2:
                mycatlist.append(4.6)


            # 5 CATEGORY: bad?
            else:
                mycatlist.append(5)


        except:
            if 'parents' in locals():
                myparentlist.append(parents)
            else:
                myparentlist.append(mab_id_list)
            mysubclonelist.append(mab_id_list)
            # 0 CATEGORY: These are ok. No sublone in them
            # check if all entries have no dots.. do not include in subsequence analysis
            if sum([1 for i in mab_id_list if '.' in i])<1:
                mycatlist.append(0)    

            # 1 CATEGORY: These are ok as well. All IDs match.
            # check if all the same
            elif len(set(mab_id_list))==1:
                mycatlist.append(1)

            # -1 make my own category for failed ones
            else:
                mycatlist.append(-1)    
    return mycatlist, myparentlist, mysubclonelist


In [135]:
mytemp = get_category(data)

print(len(mytemp[0]), len(mytemp[1]), len(mytemp[2]))
mycatlist_hc = mytemp[0]
myparentlist_hc = mytemp[1]
mysubclonelist_hc = mytemp[2]

P38_H10, P41_A10, P41_G5, P41_G8, P41_H10, P52_G4, P56_B10, P60_B4, P65_F8, P65_G6, P76_G9, PDP1_A10, PDP1_A11, PDP1_A5, PDP1_A8, PDP1_B10, PDP1_B11, PDP1_B6, PDP1_D3, PDP1_D4, PDP1_D6, PDP1_E11, PDP1_F12, PDP1_F2, PDP1_F4, PDP1_G2, PDP1_G9, PDP1_H10, PDP1_H3, PDP2_A8, PDP2_B8, PDP2_C3, PDP2_C4, PDP2_C7, PDP2_C8, PDP2_D6, PDP2_E11, PDP2_E2, PDP2_E8, PDP2_F10, PDP2_F7, PDP2_G1, PDP2_G10, PDP2_G8, PS2_A4, PS2_D7, PS2_F4, PS2_F5, PS2_G1, PS2_G12
ERROR: there is an issue with P76_G9
['N485/87.1', 'N187/33.1', 'N187/3.1', 'N131A/19.2', 'L128/124.1', 'L113/12', 'L117/131', 'L122/57', 'N34/29', 'L122/143', None, 'N279/58.4', 'N279/16.1', 'N279/16.4', 'N279/58.5', 'N416/12.1', 'L118/45.2', 'L114/13.6', 'N121/2.1', 'N121/1.2', 'N440/13.1', 'N222/40.1', 'N386/100.1', 'K14/39.1', 'N282A/13.1', 'N387/20.2', 'N457/10.1', 'N147A/13.1', 'N153/6.2', 'L78/49.1', 'L112/22.1', 'N319/13.3', 'N483/108.1', 'L98/12.1', 'L112/26.1', 'N345/51.1', 'N93/41.3', 'N260/14.1', 'L103/15.1', 'N295B/66.1', 'L69/47.1', 

['N308/48.1', 'N355A/109.2', 'N355A/109.3', 'L124/41.5', 'L124/114.4', 'L124/114.5', 'L124/114.6', 'L124/114.7', 'L124/114.8', 'L124/35.9', 'L124/4.2', 'L124/4.4', 'L124/4.3', 'L124/4.5', 'N355A/90.10', 'N355A/90.4', 'N355A/90.7', 'N308/48.7', 'N308/48.8', 'N308/48.9', 'N308/48.1', 'N308/48.2', 'N308/48.3', 'N308/48.4', 'N308/48.5', 'N308/48.6', 'N308/48.10']
P6_A3, P6_A8, P6_B7, P6_C6, P6_D6, P6_E2, P6_F2, P6R_E7, P6R_G7, P6R_H1, P6R_H6
['L124/107.1.3', 'L124/107.4', 'L124/41.1.3', 'N308/48.3', 'L124/35.8', 'N355A/109.4', 'L124/114.5', 'N355A/90.3', 'N355A/90.8', 'N355A/90.10', 'N355A/90.9']
P6_A8, P6_B7, P6_C6, P6_D10, P6_D11, P6_D12, P6_E11, P6_E7, P6_E8, P6_E9, P6_H1, P6_H2, P6R_A12, P6R_D2, P6R_D3, P6R_D5, P6R_D6, P6R_F8, P6R_G6
['L124/107.4', 'L124/41.1.3', 'N308/48.3', 'N355A/109.5', 'N355A/109.6', 'N355A/109.2', 'L124/114.2', 'N355A/90.3', 'L124/41.5', 'L124/41.6', 'N355A/90.10', 'N355A/90.4', 'L124/41.1.6', 'L124/35.3', 'L124/35.4', 'L124/35.7', 'L124/35.8', 'L124/35.9', 'L124

['L121/51', 'L121/66', 'L121/93']
P59_E12, P59_F7, P60_C6
['L122/22', 'L122/31', 'L122/73']
P38_H9, P70_A3, P70_A4, P75_G12, PDP2_D6
ERROR: there is an issue with P75_G12
['N485/20.1', 'N485/22.4.4', 'N485/22.4.5', None, 'N345/51.1']
PDP2_B8, PDP2_C3, PDP2_G10
['L112/22.1', 'N319/13.3', 'N3/26.1']
P77_D12, PDP2_D11, PDP2_E2
ERROR: there is an issue with P77_D12
[None, 'N93/6.5', 'N260/14.1']
P72_C8, P74_G5, P74_H5, PDP2_D4
ERROR: there is an issue with P74_G5
ERROR: there is an issue with P74_H5
['N253/32.1', None, None, 'N429/47.1.1']
P3_C11, P72_C12, P76_H12, P77_H12
ERROR: there is an issue with P76_H12
ERROR: there is an issue with P77_H12
['N75/3.2', 'N75/3.2', None, None]
P2_H5, P2_H6, P65_E12
['L118/66.4', 'L118/80.2.2', 'L118/80']
P3_C3, P3_H3, P74_E3
ERROR: there is an issue with P74_E3
['N414/25.1.1', 'N474/27.1.1', None]
P29_H8, P3_D1, P3_F4
['N459/94.1', 'N447/24.8', 'N459/94.3']
P2_A1, P4_A1, P68_F9
['positivecontrol1_N263/31.1', 'positivecontrol1_N263/31.1', 'N263/31.3']


['N185/6.2', 'N185/6.3', 'N185/6.6', 'N185/6.5', 'N185/7.2', 'N185/6.1', 'N185A/7.2', 'N185/7.3']
P1R_G12, P27_F2, P27_F3, P27_F4, P75_F2
ERROR: there is an issue with P75_F2
['N124/12.2', 'N134/12.1', 'N134/12.3', 'N134/12.4', None]
P1R_G12, P27_F2, P27_F3, P27_F4, P75_F2
ERROR: there is an issue with P75_F2
['N124/12.2', 'N134/12.1', 'N134/12.3', 'N134/12.4', None]
P37_E1, P37_E2
['positivecontrol2', 'N323A/52.1']
P37_E11, P37_H5
['K35/43.2', 'K35/40.2']
P38_C2, P4_B6, P69_B10, P69_B9
['N382/74.1', 'N382/14.1', 'N382/14.3', 'N382/14.2']
P39_G4, P39_G8
['L30/26.1', 'L30/8.1']
P39_G11, P39_H1
['K81/15.1', 'K82/38.6']
P37_A1, P40_A1, P67_A10, P67_A11
['positivecontrol1_L42/17.1', 'positivecontrol1_L42/17.1', 'L42/17.2', 'L42/17.3']
P39_F1, P40_F1
['K91/20.1', 'L67/53.1']
P39_G1, P40_G1
['L3/19.1', 'L63/30.1']
P40_H2, P40_H3
['L85/23.1', 'L85/3.1']
P40_H4, P40_H5
['L95A/60.1', 'L95A/61.1']
P40_E7, P40_G4
['L85/39.1', 'L85/44.1']
P40_A10, P40_A6
['L62/1.4', 'L60/14.1']
P41_E2, P41_E3
['N2

['N97A/23.1', 'N101/23.1', 'N97A/23.1']
PS1_B2, PS1_E4
['L107/24.1', 'L111/129.1']
P24_D10, PS3_D8
['N361/12.1', 'N366/12.1']
P32R_D2, P32R_D3
['N252A/14.1', 'N101/22.1']
P32R_C3, P32R_H3
['N233/22.1', 'N237/19.1']
P34_F9, P67_F12, P67_G1
['N129/26.1', 'N129/15.2', 'N129/15.3']
P67_C4, P67_C5, P73_C2, PDP2_G5
ERROR: there is an issue with P73_C2
['K51/1.2', 'K51/1.3', None, 'N293/50.1.3']
P2_F1, P67_G8, P73_E3, P73_G1
ERROR: there is an issue with P73_E3
ERROR: there is an issue with P73_G1
['N123/48.1', 'N123/48.2', None, None]
P11_A6, P67_H12, P6R_H12
['N18/22.5', 'positivecontrol_N18/22.1', 'positivecontrol_N18/22.2']
P69_E4, P69_E5, PDP2_D3
['N293/50.1.1', 'N293/50.1.4', 'N295B/78.1.1']
P46_F11, P6R_G2
['L124/2', 'L124/4.4']
P6_D2, P6R_E11, P6R_F3, P6R_F4
['L124/35.3', 'L124/114.2', 'L124/114.6', 'L124/114.7']
P6_H4, P6R_A8
['N355A/90.6', 'L124/107.4']
P70_A6, P70_A7, P76_C3, P76_H3
ERROR: there is an issue with P76_C3
ERROR: there is an issue with P76_H3
['N56/9.3', 'N56/9.4', Non

...

['N447/41.2', 'N447/41.3', 'N447/41.2']
P24_A6, P24_A7, P24_A8, P66_A7, PS3_C10
['K19/36.3', 'K19/36.4', 'K19/36.5', 'K19/36.1', 'K19/36.3']
P2_D5, P24_B10, P24_B11, P24_B7, P24_B8, P24_B9, PS3_D1
['L23/27.1', 'L23/27.5', 'L23/27.6', 'L23/27.2', 'L23/27.3', 'L23/27.4', 'L23/27.2']
P24_F6, P24_F7, P25_D8, PS3_E1, PS3_F11
['N372B/59.1.1', 'N372B/59.1.2', 'N372B/59.2', 'N372B/59.1.1', 'N372B/59.2']
P24_D12, P24_D8, P3_B10
['N361/60.4', 'N361/60.3', 'N366/60.1']
P24_E7, P24_E8, PS3_D9
['N372/1.1', 'N372/1.2', 'N372/1.1']
P24_F8, P24_F9, P25_D6, P25_D7, P3_A9, PS3_E2, PS3_F10
['N372B/60.1', 'N372B/60.3', 'N372B/60.4', 'N372B/60.5', 'N372B/60.2', 'N372B/60.1', 'N372B/60.4']
P24_H7, P24_H8, PS3_E9
['N447/47.2', 'N447/47.5', 'N447/47.2']
P24_E10, P24_E9, PS3_D10
['N372/6.2', 'N372/6.1', 'N372/6.1']
P24_F10, P24_F11, P24_F12, P24_G1, P25_B11, PS3_E3, PS3_F5
['N372A/21.2', 'N372A/21.3', 'N372A/21.4', 'N372A/21.5', 'N372A/21.1', 'N372A/21.2', 'N372A/21.1']
P24_E11, P24_E12, PS3_D11
['N372/32.1', 

['L129/96.1.1', 'L129/96.1.4', 'L129/96.1.5']
P67_H7, P67_H8
['L102/45.1.3', 'L102/45.1.4']
P66_E1, P67_F9, P74_E1
ERROR: there is an issue with P74_E1
['N124B/38.1.3', 'N124B/38.1.4', None]
P4_E7, P67_G10, P67_G9
['N104/32.1', 'N104/32.4', 'N104/32.3']
P66_H1, P67_B10, P67_B11
['L8/15.1', 'L8/15.2', 'L8/15.3']
P66_H6, P67_C10, P67_C11
['K72/8.6', 'K72/8.7', 'K72/8.8']
P4_B1, P67_D10, P67_D11
['N4/15.1', 'N4/15.5', 'N4/15.4']
P2_F6, P67_E10, P67_E11
['L94/54.4.1', 'L94/54.4', 'L94/54.5']
P2_F6, P67_E10, P67_E11
['L94/54.4.1', 'L94/54.4', 'L94/54.5']
P67_H10, P67_H9
['L128/43.1.5', 'L128/43.1.4']
P67_F10, P67_F11, P73_C11
ERROR: there is an issue with P73_C11
['L83/11.1', 'L83/11.2', None]
P67_F10, P67_F11
['L83/11.1', 'L83/11.2']
P4_E8, P67_G11, P67_G12
['N119/44.1', 'N119/44.2', 'N119/44.3']
P67_B12, P67_C1
['L62/29.2', 'L62/29.3']
P67_C12, P67_D1
['L20/8.3', 'L20/8.4']
P4_G6, P68_D1, P68_D2
['N241A/34.1', 'N241A/34.2', 'N241A/34.3']
P2_G4, P68_F12, P68_G1
['N219/5.1', 'N219/5.2', 'N2

['L61C/7.1', 'L61C/7.2', 'L61C/7.3']
P71_D6, P75_B8, PDP1_D12
ERROR: there is an issue with P75_B8
['N473/36.1', None, 'N473/36.3']
P3_A3, P71_E7
['N385/24.2.4', 'N385/24.2.2']
P65_D6, P71_B8, P71_B9
['D4/27.1', 'D4/27.2', 'D4/27.3']
P31_C12, P71_C8, P71_C9
['L61C/25.1', 'L61C/25.2', 'L61C/25.3']
P65_E8, P71_D10, P71_D9
['D4/152.1', 'D4/152.4', 'D4/152.3']
P37_H6, P71_B10, P71_B11
['L131/17.2', 'L131/17.1', 'L131/17.4']
P31_D2, P71_C10
['L61C/30.1', 'L61C/30.2']
P4_H8, P72_B1
['N304B/115.1', 'N304B/115.1']
P66_C3, P72_C1, P74_B7
ERROR: there is an issue with P74_B7
['N205B/22.1', 'N205B/22.1', None]
P66_F11, P72_E1, P74_B11
ERROR: there is an issue with P74_B11
['N151/3.3', 'N151/3.3', None]
P2_D11, P72_F1, P75_A9
ERROR: there is an issue with P75_A9
['N105/17.1', 'N105/17.1', None]
P1R_G3, P72_G1
['N37A/10.1', 'N37A/10.1']
P4_D2, P72_B2
['N145/20.1', 'N145/20.1']
P4_H7, P72_C2
['N302/10.1', 'N302/10.1']
P66_C12, P72_D2
['N94/17.1', 'N94/17.1']
P66_H3, P72_E2
['N127/31.1', 'N127/31.1']

ERROR: there is an issue with P74_H10
['N105/13.1', None, None, None]
P74_F8, P74_G2
ERROR: there is an issue with P74_F8
ERROR: there is an issue with P74_G2
[None, None]
P3_G1, P74_G8, P74_H8
ERROR: there is an issue with P74_G8
ERROR: there is an issue with P74_H8
['N1/12.1', None, None]
P74_G8, P74_H8
ERROR: there is an issue with P74_G8
ERROR: there is an issue with P74_H8
[None, None]
P74_E9, P74_F3
ERROR: there is an issue with P74_E9
ERROR: there is an issue with P74_F3
[None, None]
P1R_B6, P74_A10, P74_A12
ERROR: there is an issue with P74_A10
ERROR: there is an issue with P74_A12
['N25/35.1', None, None]
P74_D10, P74_D3
ERROR: there is an issue with P74_D10
ERROR: there is an issue with P74_D3
[None, None]
P72_F8, P74_A11
ERROR: there is an issue with P74_A11
['N15/4.1', None]
P68_F6, P74_E11
ERROR: there is an issue with P74_E11
['N173B/13.2', None]
P38_E3, P74_F11
ERROR: there is an issue with P74_F11
['N183/15.3', None]
P74_B12, P74_B8
ERROR: there is an issue with P74_B12

ERROR: there is an issue with P78_E4
['N443/75.1', None]
P34_A5, P78_B5
ERROR: there is an issue with P78_B5
['N103/1.1', None]
P31_E2, P78_D5
ERROR: there is an issue with P78_D5
['N44A/71.1', None]
P34_F10, P78_F5
ERROR: there is an issue with P78_F5
['N133/8.1', None]
P78_C4, P78_G5
ERROR: there is an issue with P78_C4
ERROR: there is an issue with P78_G5
[None, None]
P38_A7, P78_B6
ERROR: there is an issue with P78_B6
['N446/36.1', None]
P31_E8, P78_D6
ERROR: there is an issue with P78_D6
['N45B/43.1', None]
P33_G11, P78_E6
ERROR: there is an issue with P78_E6
['N457/28.1', None]
P33_G12, P78_E7
ERROR: there is an issue with P78_E7
['N457/79.1', None]
P78_A8, PDP1_E4
ERROR: there is an issue with P78_A8
[None, 'K9/68.1']
P30_C7, P78_B8
ERROR: there is an issue with P78_B8
['N375/40.2', None]
P65_A7, P78_C8
ERROR: there is an issue with P78_C8
['N481/3.1', None]
P65_B3, P78_D8
ERROR: there is an issue with P78_D8
['N481/85.1', None]
P65_C6, P78_F8
ERROR: there is an issue with P78_F

In [136]:
# TODO fix the MabID here too
data['HC_category'] = mycatlist_hc
data['HC_parents'] = myparentlist_hc
data['HC_subclone'] = mysubclonelist_hc

In [137]:
# get categories that these fall into as well (just see which ones show up a lot)
mydictcount = {}
for index,row in data.iterrows():
    mab_id_list = [plate_to_mab(i.replace(" ","")) for i in row['DuplicatedIn'].split(',')] 
    for mab_id in mab_id_list:
        if mab_id in mydictcount.keys():
            mydictcount[mab_id] +=1
        else:
            mydictcount[mab_id]=1

{k: v for k, v in sorted(mydictcount.items(), key=lambda item: item[1], reverse=True)}

ERROR: there is an issue with P76_G9
ERROR: there is an issue with P76_G3
ERROR: there is an issue with P73_H4
ERROR: there is an issue with P75_B9
ERROR: there is an issue with P74_C10
ERROR: there is an issue with P77_F7
ERROR: there is an issue with P77_G7
ERROR: there is an issue with P78_A3
ERROR: there is an issue with P73_B8
ERROR: there is an issue with P73_B9
ERROR: there is an issue with P73_C10
ERROR: there is an issue with P73_H11
ERROR: there is an issue with P77_A11
ERROR: there is an issue with P77_A8
ERROR: there is an issue with P77_F7
ERROR: there is an issue with P77_G7
ERROR: there is an issue with P74_C2
ERROR: there is an issue with P73_C12
ERROR: there is an issue with P73_G2
ERROR: there is an issue with P73_G9
ERROR: there is an issue with P73_A1
ERROR: there is an issue with P73_B1
ERROR: there is an issue with P74_G4
ERROR: there is an issue with P74_H4
ERROR: there is an issue with P73_H6
ERROR: there is an issue with P73_H8
ERROR: there is an issue with P73

ERROR: there is an issue with P74_A11
ERROR: there is an issue with P74_E11
ERROR: there is an issue with P74_F11
ERROR: there is an issue with P74_B12
ERROR: there is an issue with P74_B8
ERROR: there is an issue with P74_B10
ERROR: there is an issue with P74_F12
ERROR: there is an issue with P74_C11
ERROR: there is an issue with P74_G12
ERROR: there is an issue with P75_A1
ERROR: there is an issue with P75_A2
ERROR: there is an issue with P75_D1
ERROR: there is an issue with P75_D2
ERROR: there is an issue with P75_E2
ERROR: there is an issue with P75_G2
ERROR: there is an issue with P75_G3
ERROR: there is an issue with P75_B3
ERROR: there is an issue with P75_B4
ERROR: there is an issue with P75_B5
ERROR: there is an issue with P75_B3
ERROR: there is an issue with P75_B4
ERROR: there is an issue with P75_E4
ERROR: there is an issue with P75_G10
ERROR: there is an issue with P75_G4
ERROR: there is an issue with P75_H3
ERROR: there is an issue with P75_H4
ERROR: there is an issue with

{None: 469,
 'positivecontrol1_N222/6.2.9': 10,
 'positivecontrol1_N29/29.1': 7,
 'N440/13.1': 6,
 'positivecontrol1_L130/1': 6,
 'N355A/109.4': 6,
 'N355A/109.2': 6,
 'N355A/90.10': 6,
 'N355A/90.4': 6,
 'L119/17': 6,
 'N382/22.1': 6,
 'RNA_20% N486/76.1+20%L71/5.5+20%L130/1.1+20%N470/22.1+20%N100/13.1': 5,
 'L124/4.1': 5,
 'L124/107.1.3': 5,
 'positivecontrol1_N52A/42.1': 5,
 'positivecontrol1_N91/27.1': 5,
 'L119/35': 5,
 'K42/21.12': 5,
 'N457/10.1': 4,
 'N147A/13.1': 4,
 'L78/49.1': 4,
 'L121/82': 4,
 'L121/112': 4,
 'L124/4.4': 4,
 'N355A/90.7': 4,
 'N308/48.3': 4,
 'L124/107.4': 4,
 'L124/35.8': 4,
 'N355A/90.6': 4,
 'L111/129.3.1': 4,
 'positivecontrol_N18/22.1': 4,
 'L119/134.1': 4,
 'N278/16.1': 4,
 'positivecontrol1_L42/17.1': 4,
 'L109/50': 4,
 'L122/104': 4,
 'L122/103': 4,
 'L119/47': 4,
 'L95A/44.1': 4,
 'N110/29.1': 4,
 'N278/19.1': 4,
 'L28/4.2': 4,
 'L57/47.1': 4,
 'K65/36.1': 4,
 'K65A/30.1': 4,
 'K65/31.1': 4,
 'N420/24.1': 4,
 'N55/28.1': 4,
 'N424/62.1': 4,
 'N112

In [138]:
data.head()

,ASV,DuplicatedIn,MabIDs,ParentIDs,MabCount,ParentCount,HC_category,HC_parents,HC_subclone
0,GGTCAGCGTCAGCCCATAGTAACCTCGGGAACCCCGCTTCAAAATG...,"P38_H10, P41_A10, P41_G5, P41_G8, P41_H10, P52...","K14/39.1, L103/15.1, L108/90.1, L112/22.1, L11...","K14/39, L103/15, L108/90, L112/22, L112/26, L1...",50,48,-1.0,"[87, 33, 3, 19, 124, 12, 131, 57, 29, 143, 58,...","[N485/87.1, N187/33.1, N187/3.1, N131A/19.2, L..."
1,GTCAAAACGGCTTCACAAATCTTTTTCAAGACCACTTTCTGAGTAT...,"P40_B4, P41_D6, P41_G8, P76_G3, PDP1_A8, PDP1_...","L113/80.2, L114/13.6, L118/45.2, L132/18.4, L2...","L113/80, L114/13, L118/45, L132/18, L24/24, L3...",36,36,5.0,"[36, 49, 19, 58, 12, 45, 13, 80, 21, 13, 38, 1...","[L39/36.1A, N163A/49.1, N131A/19.2, N279/58.5,..."
2,ATATGAACACTGTTTTCTCTACAGTCACTGAATCTCAAGGTCCTTA...,"P2_E1, P5_A12, P5_B1, P5_B2, P5_B3, P65_H1, P6...",10% N48676.1 + 30% L120/12.1 + 60% L71/5.5 cel...,"10% N48676, 20% N486/76, 25% N486/76, 60% N486...",33,24,5.0,"[76, 76, 76, 76, 76, 76, 76, 12, 76, 76, 76, 7...","[N486/76.1, N486/76.5, N486/76.4, N486/76.3, N..."
3,AGCTCTGACAGAGGAGGCCGGTCCTGGATTCGATTCCCAGTTCCTC...,"P2_G1, P65_H1, P65_H10, P65_H2, P65_H3, P65_H4...",10% N48676.1 + 30% L120/12.1 + 60% L71/5.5 cel...,"10% N48676, 20% N486/76, 25% N486/76, 60% N486...",21,16,5.0,"[12, 76, 76, 12, 76, 76, 76, 76, 12, 76, 76, 7...","[L120/12.1, 25%N486/76.1+25%L120/12.1+50%L71/5..."
4,AAAACATGAGATCACAGTTCTCTCTACAGTTACTGAGCACACAGGA...,"P18_F4, P18_F6, P18_F7, P2_C3, P65_H1, P65_H10...",10% N48676.1 + 30% L120/12.1 + 60% L71/5.5 cel...,"10% N48676, 20% N486/76, 25% N486/76, 60% N486...",21,15,5.0,"[5, 5, 5, 5, 76, 76, 12, 76, 12, 76, 76, 76, 7...","[L71/5.1, L71/5.3, L71/5.4, L71/5.5, 25%N486/7..."


In [139]:
# export this for belvin to check out
data.to_csv('heavy_chain_cat.tsv',index=False, sep='\t')

In [140]:
dup_heav = pd.read_csv('heavy_chain_cat.tsv', sep='\t')
dup_heav[dup_heav['HC_category']==2.0]


,ASV,DuplicatedIn,MabIDs,ParentIDs,MabCount,ParentCount,HC_category,HC_parents,HC_subclone
370,ATAAGATCACTGTTCTCTCTACAGTTACTGAGCACACAGGACCTCA...,"P30_A3, P66_B12, P73_D11, P73_E10","K7/45.1, K7/45.1, K7/45.3, L76/36.6","K7/45, L76/36",4,2,2.0,"['45', '45']","['K7/45.3', 'K7/45.1']"
407,AGCTCTGACAGAGGAGGCCAGTCCTGGATTTGAGTTCCTCACATTC...,"P2_F1, P67_G8, P73_E3, P73_G1","1F1.1.1.18, N123/48.1, N123/48.1, N123/48.2","1F1, N123/48",4,2,2.0,"['48', '48']","['N123/48.1', 'N123/48.2']"
413,GACATATGACCAATATCCTCTCCACATTCCCTTAAGACACTGACTC...,"P70_A6, P70_A7, P76_C3, P76_H3","Empty for negative control, N56/9.1, N56/9.3, ...","Empty for negative control, N56/9",4,2,2.0,"['9', '9']","['N56/9.3', 'N56/9.4']"
415,ATCCTCTTCTCATAGAGCCTCCATCAGAGCATGGCTGTCTTGGGGC...,"P2_F3, P70_B12, P74_E2, P74_H10","N105/13.1 + N408/79.9, N408/79.10, N408/79.9, ...","N105/13, N408/79",4,2,2.0,"['79', '79']","['N408/79.9', 'N408/79.10']"
416,ACAGTCCCTGAAGACACTGACTCTAACCATGGAATGGAGCTGGGTC...,"P29_G4, P71_D4, P71_D5, P78_H5","L100/1.1, L100/1.2, L100/1.3, Positive control...","L100/1, Positive control L100/1",4,2,2.0,"['1', '1', '1']","['L100/1.1', 'L100/1.2', 'L100/1.3']"
...,...,...,...,...,...,...,...,...,...
1048,AGCTCTGACAGAGGAGCCAAGCCCTGGATTCCCAGGTCCTCACATT...,"P65_D6, P71_B8, P71_B9","D4/27.1, D4/27.2, D4/27.3",D4/27,3,1,2.0,"['27', '27', '27']","['D4/27.1', 'D4/27.2', 'D4/27.3']"
1049,ATCCTCTTCTCATAGAGCCTCCATCAGAGCATGGCTGTCCTGGGGC...,"P31_C12, P71_C8, P71_C9","L61C/25.1, L61C/25.2, L61C/25.3",L61C/25,3,1,2.0,"['25', '25', '25']","['L61C/25.1', 'L61C/25.2', 'L61C/25.3']"
1050,AACCGACGATCAGTGTCCTCTCCAAAGTCCCTGAACACACTGACTC...,"P65_E8, P71_D10, P71_D9","D4/152.1, D4/152.3, D4/152.4",D4/152,3,1,2.0,"['152', '152', '152']","['D4/152.1', 'D4/152.4', 'D4/152.3']"
1051,ATATGAGCCCTATCTTCTCTACAGACACTGAATCTCAAGGTCCTTA...,"P37_H6, P71_B10, P71_B11","L131/17.1, L131/17.2, L131/17.4",L131/17,3,1,2.0,"['17', '17', '17']","['L131/17.2', 'L131/17.1', 'L131/17.4']"


# LIGHT CHAINS

In [147]:
import pandas as pd
# why does the MabID column have different number of values then the duplicated in
data = pd.read_csv('DuplicatedLC_ASVs.tsv', sep='\t')
data.tail()

,ASV,DuplicatedIn,MabIDs,ParentIDs,MabCount,ParentCount
1677,ACTGATCACTCTCCTATGTTCATTTCCTCAAAATGATGAGTCCTGC...,"P29_H6, P78_G10, P78_G11","N459/90.1, N459/90.2, N459/90.5",N459/90,3,1
1678,ACCAAAATTCAAAGAGAAAATGGATTTTCAGGTGCAGATTTTCAGC...,"P29_H6, P78_G10, P78_G11","N459/90.1, N459/90.2, N459/90.5",N459/90,3,1
1679,GAAATACATCAGACCAGCATGGGCATCAAGATGGAGACACATTCTC...,"P31_B5, P78_C11","N439/39.1, N439/39.3",N439/39,2,1
1680,ACTGATCAGTCTCCTCAGGCTGTCTCCTCAGGTTGCCTCCTCAAAA...,"P72_G3, PRnD_N13","N15/39.1, N15/39.1",N15/39,2,1
1681,AATCAGAACTCTTCACACCAAGTTCTCAGAATGAGGTGCTCTCTTC...,"P72_H7, PRnD_N17","N271/44.1, N271/44.1",N271/44,2,1


In [145]:

mycatlist_lc = get_category(data)
print(len(mycatlist_lc))
data['mycat'] = mycatlist_lc


P10_C11, P10_C12, P10_C5, P10_C8, P10_D1, P10_D3, P10_D8, P10R_C11, P10R_C12, P10R_C5, P11_F3, P11_G10, P11_G9, P11_H5, P14_B4, P14_B8, P14_C4, P14_F10, P14_F12, P14_G2, P14_G6, P14_G7, P14_G8, P17_C4, P17_C6, P17_C7, P17_C9, P18_G6, P19_B2, P1R_F6, P1R_H5, P20_B2, P20_E9, P20_F10, P20_G4, P21_C7, P21_D4, P21_E6, P22_F10, P22_G8, P23_A2, P23_A3, P23_A4, P23_B10, P23_B8, P27_D6, P27_H5, P28_A8, P28_B5, P28_B6, P28_D12, P28_G10, P28_G5, P28_G8, P28_H1, P28_H5, P28_H9, P3_H9, P30_G1, P32R_A12, P32R_B10, P32R_C10, P32R_D8, P32R_E1, P32R_E2, P32R_E8, P33_B10, P33_C10, P33_C9, P33_E3, P36_B10, P36_H6, P37_A10, P37_A7, P37_A8, P37_B2, P37_B3, P37_B4, P37_B6, P37_B7, P37_E4, P37_G6, P37_H4, P37_H5, P38_B4, P38_D10, P38_D6, P38_E10, P38_E6, P38_G3, P38_H1, P38_H10, P38_H2, P38_H4, P38_H5, P39_C11, P39_C6, P39_E11, P39_E4, P39_F10, P39_G12, P39_H6, P4_E10, P41_A10, P41_A12, P41_A7, P41_B10, P41_C3, P41_C5, P41_E10, P41_F1, P41_F5, P41_G3, P41_G8, P42_B9, P42_C6, P42_D5, P42_F3, P42_G11, P42_G3, 

['L119/106.8', 'N373/49.1', 'N261/14.1', 'negativecontrol1', 'N485/87.1', 'L39/36.1A', 'L95B/22.1', 'N163/6.1', 'N187/3.1', 'K20/2', 'L120/36', 'L120/39', 'L121/94', 'L121/96', 'L122/143', None, 'N279/58.4', 'N416/12.1', 'N121/2.1', 'N121/1.2', 'N440/13.1', 'N374/38.1', 'K14/39.1', 'N417/72.1', 'N267/51.3', 'N164/11.2', 'L78/49.1', 'N295A/1.1', 'L112/22.1', 'N454/91.1', 'L112/26.1', 'D4/176.1', 'N93/6.5', 'N345/51.1', 'N175/5.7', 'N93/41.3', 'N260/14.1', 'L69/47.1', 'N292/15.1', 'L111/129.3.1', 'N420/26.1', 'N420/40.1', 'N410/17.1']
P10_G4, P10_G5, P10_G6, P10_G7, P10_G8, P10_H1, P10_H2, P10R_G4, P10R_G5, P10R_G6, P10R_G8, P16_B1, P16_F3, P16_H3, P17_F11, P17_G2, P17_G5, P20_D12, P20_E1, P20_E2, P20_E3, P20_E4, P34_E3, P39_F4, P40_C2, P43_B12, P43_F1, P43_H2, P43_H4, P44_D11, P44_G1, P45_B4, P45_D7, P46_B7, P46_H6, P47_A10, P49_C12, P49_G7, P50_C1, P50_G2, P50_G7, P51_C12, P51_D5, P51_F10, P51_F8, P52_G7, P52_G9, P55_D7, P55_E7, P55_F9, P56_F12, P56_H9, P57_A12, P58_F5, P60_E12, P73_G4

ERROR: there is an issue with P75_A6
ERROR: there is an issue with P75_C5
ERROR: there is an issue with P78_B10
ERROR: there is an issue with P78_B12
['N453/35.1', 'N426/40.1', 'N486/37', 'K28/18', 'L106/37', 'L124/48', 'L124/65', 'L124/118', 'L127/48', 'L124/139', 'L127/72', 'L127/80', 'L127/125', 'N450/78', 'L107/67', 'L109/37', 'L109/70', 'L108/30', 'L108/43', 'L111/22', 'L117/18', 'L117/27', 'L117/125', 'L119/99', None, None, None, None]
P38_B4, P38_D10, P38_E4, P38_E6, P38_H2, P65_A8, P65_F8, P65_G6, P67_D5, P76_G9, PDP1_B1, PDP1_C2, PDP1_D3, PDP1_D5, PDP1_D6, PDP1_E11, PDP1_F12, PDP1_G2, PDP1_G6, PDP1_G9, PDP1_H5, PS1_B6, PS1_D4, PS2_D8, PS2_F4, PS2_F5, PS2_H8
ERROR: there is an issue with P76_G9
['N479/120.1', 'N391/65.1', 'N388A/49.1', 'N483/108.1', 'N402/83.1', 'N488/58.1', 'N34/29', 'L122/143', 'K91/36.4', None, 'N279/16.5', 'L113/80.2', 'N121/2.1', 'N121/21.1', 'N440/13.1', 'N222/40.1', 'N386/100.1', 'N387/20.2', 'N267/5.1', 'N457/10.1', 'N142A/33.1', 'L113/80.1', 'L107/50.1

['L124/15', 'L124/50', 'L124/52', 'L124/94', 'L127/54', 'L109/18', 'L109/35', 'L108/23', 'L117/74', 'L117/134', 'L121/10', 'L122/30', 'N402/34.4.1', 'N451/33.1']
P3_D3, P71_E11, P71_F1, P71_F10, P71_F3, P71_F4, P71_F6, P71_F9, P71_G1, P71_G11, P71_G12, P71_G3, P71_G4, P71_G6, P71_G9, P75_B9
ERROR: there is an issue with P75_B9
['N100/13.1', 'N100/13.1', 'cells_50% N486/76.1 + 50% N100/13.1', 'cells_50% N100/13.1 + 50% N470/22.1', 'cells_50% L71/5.5 + 10% N486/76.1 + 40% N100/13.1', 'cells_80% N100/13.1 + 20% L130/1.1', 'cells_60% N100/13.1 + 20% N486/76.1 + 20% N470/22.1', 'cells_25% N486/76.2 + 25% N100/13.1 + 25% N470/22.1 + 25% L71/5.5', 'RNA_50% N486/76.1 + 50% N100/13.1', 'RNA_50% N100/13.1+50%N470/22.1', 'RNA_20% N486/76.1+20%L71/5.5+20%L130/1.1+20%N470/22.1+20%N100/13.1', 'RNA_50% L71/5.5 + 10% N486/76.1 + 40% N100/13.1', 'RNA_80% N100/13.1 + 20% L130/1.1', 'RNA_60% N100/13.1 + 20% N486/76.1 + 20% N470/22.1', 'RNA_25% N486/76.2+ 25% N100/13.1+N470/22.1+L71/5.5', None]
P43_G12, P

['L118/57', 'L124/21', 'L124/75', 'L124/67', 'L124/101', 'L127/20', 'L109/56', 'L109/76', 'L109/80']
P43_A2, P46_G9, P46_H10, P47_A3, P48_D2, P49_H8, P50_F10, P51_A7, P56_G4
['N29/35', 'L124/13', 'L124/26', 'L124/28', 'L127/45', 'L127/129', 'N450/89', 'L109/12', 'L119/84']
P47_D4, P47_F1, P48_A6, P48_F9, P50_F11, P52_G12, P56_B2, P57_D1, P78_G3
ERROR: there is an issue with P78_G3
['L124/69', 'L124/89', 'L127/9', 'L124/139', 'N450/91', 'L113/21', 'L117/116', 'L120/19', None]
P46_E12, P55_H2, P56_H4, P57_C9, P58_D8, P59_A11, P59_B5, P59_H6, P60_B8
['L106/78', 'L117/100', 'L119/98', 'L120/15', 'L120/120', 'L121/60', 'L121/66', 'L121/117', 'L122/61']
P41_C5, PDP2_B6, PDP2_C2, PDP2_E1, PDP2_F4, PDP2_F7, PDP2_G1, PS2_F4, PS2_F5
['N163/6.1', 'N485/22.4', 'N289/39.1', 'N175/5.7', 'N402/83.1', 'L69/47.1', 'N193A/10.1', 'N420/26.1', 'N420/40.1']
P31_A11, P31_A12, P31_B1, P46_H2, P6_A1, P6_A2, P6_B2, P6R_G11, P77_C5, P77_C6, P78_H7, P78_H8
ERROR: there is an issue with P77_C5
ERROR: there is an 

...

['N387/1.2', 'N387/1.2']
P66_G2, PS3_F3
['N16B/8.1', 'N16B/8.2']
P24_G6, PS3_E5
['N372B/51.4', 'N372B/51.4']
P23_D6, PS3_B6
['K60/41.2', 'K60/41.2']
P24_G10, PS3_E6
['N447/23.2', 'N447/23.2']
P22_H10, PS3_A7
['N420/64.6.1', 'N420/64.6.1']
P23_B2, PS3_A11
['K60/35.2', 'K60/35.1']
P25_D8, PS3_F11
['N372B/59.2', 'N372B/59.2']
P32R_B3, P32R_B7
['N379/52.1', 'N379/52.1.1']
P32R_H5, P32R_H7
['N248/32.1', 'N248/32.2']
P32R_B4, P32R_B8
['N379/72.1', 'N379/72.1.1']
P66_A10, P67_A12, P67_B1
['L5/1.1', 'L5/1.2', 'L5/1.3']
P1R_C8, P67_B12, P67_C1
['L62/29.1', 'L62/29.2', 'L62/29.3']
P4_D3, P67_D12, P67_E1
['N4/49.1', 'N4/49.3', 'N4/49.2']
P1R_F11, P67_F12, P67_G1
['N129/15.1', 'N129/15.2', 'N129/15.3']
P1R_F11, P67_F12, P67_G1
['N129/15.1', 'N129/15.2', 'N129/15.3']
P66_B9, P67_H1, P67_H2
['N111/24.1', 'N111/24.2', 'N111/24.3']
P2_A12, P67_E2, P67_E3
['K78/29.4', 'K78/29.2', 'K78/29.3']
P2_A12, P67_E2, P67_E3
['K78/29.4', 'K78/29.2', 'K78/29.3']
P2_D3, P67_F2, P67_F3
['L115/13.3', 'L115/13.1', 'L1

['N67/15.2', 'N67/15.3', 'N67/15.1']
P1R_A3, P70_F3, P70_F4
['N92/14.1', 'N92/14.2', 'N92/14.3']
P1R_E12, P70_E4, P70_E5
['N77/15.1', 'N77/15.2', 'N77/15.3']
P4_E4, P70_G4, P70_G5
['N411/51.1.1', 'N411/51.1.2', 'N411/51.1.4']
P4_D1, P70_A5, P74_B2
ERROR: there is an issue with P74_B2
['N421A/85.1', 'N421A/85.2', None]
P2_A11, P70_C9, P70_D5, P76_C10
ERROR: there is an issue with P76_C10
['N440/21.1.1.1', 'N440/21.1.1.3', 'N440/21.1.1.2', None]
P3_B4, P70_F5, P70_F6
['N483/126.2', 'N483/126.3', 'N483/126.4']
P3_F1, P70_C6, P76_C8
ERROR: there is an issue with P76_C8
['N425/45.4', 'N425/45.7', None]
P2_H9, P70_D6, P70_D7
['N56/21.1', 'N56/21.2', 'N56/21.3']
P4_H5, P70_G6, P70_G7
['N442/28.1.1', 'N442/28.1.2', 'N442/28.1.3']
P3_F3, P70_H6, P75_E3
ERROR: there is an issue with P75_E3
['N483/84.2', 'N483/84.4', None]
P2_F9, P70_B7, P70_B8
['N479/12.1', 'N479/12.2', 'N479/12.5']
P3_E12, P70_C7, P70_C8
['N441/35.1', 'N441/35.2', 'N441/35.3']
P70_D6, P70_D7
['N56/21.2', 'N56/21.3']
P1R_B8, P70

ERROR: there is an issue with P74_B10
ERROR: there is an issue with P74_F12
[None, None, 'N143/36.11']
P3_D12, P75_A1, P75_A2
ERROR: there is an issue with P75_A1
ERROR: there is an issue with P75_A2
['N245/1.1', None, None]
P3_F12, P75_D1, P75_D2
ERROR: there is an issue with P75_D1
ERROR: there is an issue with P75_D2
['N274/8.1', None, None]
P66_C7, P75_E2
ERROR: there is an issue with P75_E2
['N88/12.1', None]
P66_C7, P75_E2
ERROR: there is an issue with P75_E2
['N88/12.1', None]
P4_B2, P75_G2, P75_G3
ERROR: there is an issue with P75_G2
ERROR: there is an issue with P75_G3
['N377/20.1', None, None]
P3_G6, P75_B3, P75_B4, P75_B5
ERROR: there is an issue with P75_B3
ERROR: there is an issue with P75_B4
ERROR: there is an issue with P75_B5
['N385/21.1', None, None, None]
P3_E2, P75_E4, P75_G10
ERROR: there is an issue with P75_E4
ERROR: there is an issue with P75_G10
['N323B/20.3.6', None, None]
P69_B6, P75_G4
ERROR: there is an issue with P75_G4
['N377/20.3', None]
P75_H3, P75_H4
ER

ERROR: there is an issue with P78_B7
['N375/11.1', None]
P33_G12, P78_E7
ERROR: there is an issue with P78_E7
['N457/79.1', None]
P78_A8, PDP1_E4
ERROR: there is an issue with P78_A8
[None, 'K9/68.1']
P30_C7, P78_B8
ERROR: there is an issue with P78_B8
['N375/40.2', None]
P65_A7, P78_C8
ERROR: there is an issue with P78_C8
['N481/3.1', None]
P65_B3, P78_D8
ERROR: there is an issue with P78_D8
['N481/85.1', None]
P65_C6, P78_F8
ERROR: there is an issue with P78_F8
['N435/7.1', None]
P29_H9, P78_G8, P78_G9
ERROR: there is an issue with P78_G8
ERROR: there is an issue with P78_G9
['N459/52.1', None, None]
P78_A9, PDP1_E2
ERROR: there is an issue with P78_A9
[None, 'K19/13.2']
P30_C12, P78_B9
ERROR: there is an issue with P78_B9
['N375/92.1', None]
P27_G1, P78_D9
ERROR: there is an issue with P78_D9
['N134/30.2.3', None]
P65_C9, P78_F9
ERROR: there is an issue with P78_F9
['N435/72.1', None]
P33_H5, P78_H9
ERROR: there is an issue with P78_H9
['N482/28.1', None]
P13_A2, P78_A10
ERROR: ther

KeyError: 'mycat'

In [ ]:
#for index,row in data.iterrows():
#    if row['mycat'] != row['DuplicatedCategory']:
#        #if row['mycat'] != 6:
#            print([row['mycat'], row['DuplicatedCategory'], row['MabIDs']])
        

In [ ]:
# get categories that these fall into as well
mydictcount = {}
for index,row in data.iterrows():
    mab_id_list = [plate_to_mab(i.replace(" ","")) for i in row['DuplicatedIn'].split(',')] 
    for mab_id in mab_id_list:
        if mab_id in mydictcount.keys():
            mydictcount[mab_id] +=1
        else:
            mydictcount[mab_id]=1

{k: v for k, v in sorted(mydictcount.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
# export this for Ipek to check out
data.head()

In [ ]:
data.to_csv('light_chain_cats.tsv',index=False, sep='\t')

# Get the sequences that are "complete"

In [ ]:
lc = pd.read_csv('light_chain_cats.tsv', sep='\t')
lc.head()

In [ ]:
hc = pd.read_csv('heavy_chain_cat.tsv', sep='\t')
hc.head()

In [ ]:
lc_2 = lc[lc['mycat']==2]
hc_2 = hc[hc['HC_category']==2]

In [ ]:
import math
lc_list = []
for i in lc_2['MabIDs']:
    if isinstance(i, str):
        split_parents = i.replace(' ','').split(',')
        for x in split_parents:
            lc_list.append(x)

hc_list = []
for i in hc_2['MabIDs']:
    if isinstance(i, str):
        split_parents = i.replace(' ','').split(',')
        for x in split_parents:
            hc_list.append(x) 

print(list(set(lc_list) ^ set(hc_list)))
print(list(set(lc_list) & set(hc_list)))

### Compare this to category 1 and 3 (neuromab category)
### Compare this to category 2 (neuromab category)

In [ ]:
metadata = pd.read_csv("../static_data/2021-03-12.tsv", sep='\t')
metadata.head()

In [ ]:
cat1 = metadata[metadata['Category']==1]['trimmer_id']
cat2 = metadata[metadata['Category']==2]['trimmer_id']
cat3 = metadata[metadata['Category']==3]['trimmer_id']
hc_lc_ok = list(set(lc_list) & set(hc_list))
#cat1_parent = ["".join(i.split('.')[0:-1]) for i in cat1]
#cat3_parent = ["".join(i.split('.')[0:-1]) for i in cat3]
cat1_parent = [i for i in cat1]
cat2_parent = [i for i in cat2]
cat3_parent = [i for i in cat3]

In [ ]:
print(len(list(set(cat1_parent) & set(hc_lc_ok))))
cat1_lc_hc_ok = list(set(cat1_parent) & set(hc_lc_ok))
print(cat1_lc_hc_ok[:5])

In [ ]:
print((len(list(set(cat3_parent) & set(hc_lc_ok)))))
cat3_lc_hc_ok =list(set(cat3_parent) & set(hc_lc_ok))
print(cat3_lc_hc_ok[:5])

In [ ]:
print((len(list(set(cat2_parent) & set(hc_lc_ok)))))
cat2_lc_hc_ok = list(set(cat2_parent) & set(hc_lc_ok))
print(cat2_lc_hc_ok[:5])

In [ ]:
final_parent_dict = {}
for mabid in cat1_lc_hc_ok + cat2_lc_hc_ok + cat3_lc_hc_ok:
    parent = mabid.split('.')[0]
    if parent in final_parent_dict.keys():
        final_parent_dict[parent].append(mabid)
    else:
        final_parent_dict[parent] = [mabid]

        
for key in final_parent_dict.keys():
    print(key + '\t' + str(len(final_parent_dict[key])) + '\t' + ', '.join(final_parent_dict[key]))

# DROPOUTS

### lets see what the possible combinations possible are

In [146]:
set(dropout['Dropout? Missing'])

KeyError: 'Dropout? Missing'

In [ ]:
set(dropout['Dropout resolution'])

In [ ]:
list(dropout['Dropout? Missing']).count('Dec2020dropout-Both')

### Lets groupby trimmer ID and then we can create some combinations of lists that are interesting.

In [ ]:
grouped = dropout.groupby('trimmer_id')['Dropout? Missing'].apply(list).reset_index()

In [ ]:
grouped['Dropout Resolution'] = dropout.groupby('trimmer_id')['Dropout resolution'].apply(list).reset_index()['Dropout resolution']

In [ ]:
list_both = []
list_both_twice = []
list_HC = []
list_LC = []
list_HC_only = []
list_LC_only = []


for index, item in grouped.iterrows():

    if item['Dropout? Missing'].count("Dec2020dropout-Both")>=1 and item['Dropout? Missing'].count("Dec2020dropout-HC")>=1:
        list_HC.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")>=1 and item['Dropout? Missing'].count("Dec2020dropout-LC")>=1:
        list_LC.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")<1 and item['Dropout? Missing'].count("Dec2020dropout-HC")>=1:
        list_HC_only.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")<1 and item['Dropout? Missing'].count("Dec2020dropout-LC")>=1:
        list_LC_only.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")>=1:
        list_both.append(index)
    if item['Dropout? Missing'].count("Dec2020dropout-Both")>=2:
        list_both_twice.append(index)


In [ ]:
grouped.ix[list_both].sort_values(by=['trimmer_id'])


In [ ]:
grouped.sort_values(by=['trimmer_id']).to_csv('dropout_groupings.tsv', sep='\t')

In [ ]:
# if there are other subclones from that we dont care about double dropouts

list([','.join(i) for i in grouped.ix[list_both_twice]['Dropout Resolution'] if type(i)==list and all(isinstance(i, str))])

In [ ]:
type('test')==str

In [ ]:
grouped.ix[list_HC]

In [ ]:
grouped.ix[list_LC]

In [ ]:
grouped.ix[list_HC_only]

In [ ]:
grouped.ix[list_LC_only]